In [1]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
from PIL import Image as im
import sklearn
from sklearn import linear_model, datasets, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import time
import tensorflow as tf
from tensorflow import keras
import nbconvert

Paramètres

In [2]:
n, m = (30, 30) #taille des images
p = 4 #nombre de classes (si on utilise toute la BDD les calculs sont longs)

Fonction qui permet de lire les fichiers des données

In [3]:
def readTrafficSigns(rootpath):
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''
    images = [] # images
    labels = [] # corresponding labels
    # loop over all 42 classes
    for c in range(0,p-1):
        prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') # annotations file
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        next(gtReader) # skip header
        # loop over all images in current annotations file
        for row in gtReader:
            images.append(plt.imread(prefix + row[0])) # the 1th column is the filename
            labels.append(row[7]) # the 8th column is the label
        gtFile.close()
    return images, labels

In [4]:
data = (readTrafficSigns("C:/Users/guill/PycharmProjects/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images"))

Nommer les colonnes et mettre le dataframe dans le bon sens

In [5]:
df = pd.DataFrame(data, ["data", "target"])
df = df.transpose()

Découpage du jeu de données

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df["data"], df["target"], test_size=0.33)
indices = X_train.index
indices2 = X_test.index

Resizing des images d'entraînement et de test

In [7]:
for i in indices:
    varimage = X_train[i].copy()
    resized = im.fromarray(varimage).resize((n,m))
    X_train[i] = np.asarray(resized)
    X_train[i] = np.ndarray.flatten(X_train[i])

In [8]:
for i in indices2:
    varimage = X_test[i].copy()
    resized = im.fromarray(varimage).resize((n,m))
    X_test[i] = np.asarray(resized)
    X_test[i] = np.ndarray.flatten(X_test[i])

Reformatage

In [9]:
X_train, X_test, y_train, y_test = X_train.tolist(), X_test.tolist(), (y_train.astype(int)).tolist(), (y_test.astype(int).tolist())

Choix du modèle (ici NN de 3 couches de 100 neurones)

In [10]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 100, 100), random_state=1, max_iter=5000)
start_time = time.time()
mlp.fit(X_train, y_train)
end_time = time.time()
print("Score sur l'entraînement :", mlp.score(X_train, y_train), "Score sur les données test :", mlp.score(X_test, y_test),"Temps d'entraînement :", end_time-start_time)

Score sur l'entraînement : 1.0 Score sur les données test : 0.970873786407767 Temps d'entraînement : 33.90434217453003


Tensorflow

Avec Sklearn, les modèles renvoient le numéro associé à la classe alors qu'en Tensorflow, les modèles renvoient la liste des probas d'appartenance à chacune des catégories. Donc on adapte $y_{train}$ et $y_{test}$.

In [69]:
y_trainflow = []
for i in range(len(y_train)):
    L = [0]*10
    L[y_train[i]-1] = 1
    y_trainflow.append(L)
y_trainflow = np.array(y_trainflow)

In [94]:
X_trainflow = np.stack(X_trainflow)

NN de deux couches de 128 neurones

In [85]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7)
])

In [86]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [87]:
model.fit(X_trainflow, y_trainflow, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\guill\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\guill\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\guill\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\guill\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\guill\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\guill\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\guill\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\guill\anaconda